In [11]:
import warnings
warnings.filterwarnings("ignore",category = FutureWarning)

import matplotlib.pyplot as plt
from builtins import range,input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np

from glob import glob
import pandas as pd
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

print("All libraries are imported")

All libraries are imported


In [2]:
#define size to which images are to be resized
IMAGE_SIZE = [250,250]

#training config
epochs = 200
batch_size = 32

#define paths
covid_path = "E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID"
noncovid_path = "E:\\PYTHON\\LetsUpgrade\\Capstone_project\\non-COVID"

#use glob to grab images from path .jgp or .jpeg
covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

In [3]:
#visualize file variable contents
print("First 5 Covid Files: ", covid_files[0:5])
print("______________________________________________________________________________________________________________________________________________________________")
print("Total Counts: ", len(covid_files))
print("______________________________________________________________________________________________________________________________________________________________")
print("First 5 NonCovid Files: ", noncovid_files[0:5])
print("______________________________________________________________________________________________________________________________________________________________")
print("Total Counts: ", len(noncovid_files))

First 5 Covid Files:  ['E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID\\Covid (1).png', 'E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID\\Covid (10).png', 'E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID\\Covid (100).png', 'E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID\\Covid (1000).png', 'E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID\\Covid (1001).png']
______________________________________________________________________________________________________________________________________________________________
Total Counts:  1252
______________________________________________________________________________________________________________________________________________________________
First 5 NonCovid Files:  ['E:\\PYTHON\\LetsUpgrade\\Capstone_project\\non-COVID\\Non-Covid (1).png', 'E:\\PYTHON\\LetsUpgrade\\Capstone_project\\non-COVID\\Non-Covid (10).png', 'E:\\PYTHON\\LetsUpgrade\\Capstone_project\\non-COVID\\Non-Covid (100).png', 'E:\\PYTHON\\LetsUpgrade\\Capstone_projec

In [4]:
#FETCH IMAGES AND CLASS LABELS FROM FILES
covid_labels = []                          #empty list for append the labels
noncovid_labels = []                       

covid_images = []                           #empty list for append the images
noncovid_images = []

for i in range(len(covid_files)):
    image = cv2.imread(covid_files[i])              #reading the file
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #arrange format as per deep learning libraries
    image = cv2.resize(image,(250,250))             #resize as per model
    covid_images.append(image)                      #append image
    covid_labels.append('CT_COVID')                 #append class label
    
for i in range(len(noncovid_files)):
    image = cv2.imread(noncovid_files[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(250,250))
    covid_images.append(image)
    covid_labels.append('CT_NonCOVID')

In [5]:
train_datagen = ImageDataGenerator(
                fill_mode = 'nearest',
                validation_split=0.2
)

In [8]:
train_generator=train_datagen.flow_from_directory(
    "E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID",
    target_size=(108,108),
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
)
validation_generator=train_datagen.flow_from_directory(
    "E:\\PYTHON\\LetsUpgrade\\Capstone_project\\COVID",
    target_size=(108,108),
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',
)

train_generator=train_datagen.flow_from_directory(
    "E:\\PYTHON\\LetsUpgrade\\Capstone_project\\non-COVID",
    target_size=(108,108),
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
)
validation_generator=train_datagen.flow_from_directory(
    "E:\\PYTHON\\LetsUpgrade\\Capstone_project\\non-COVID",
    target_size=(108,108),
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [12]:
import tensorflow as tf
from keras.applications import ResNet50
model = tf.keras.models.Sequential([
    ResNet50(input_shape=(108,108,3), include_top=False),
])
for layer in model.layers:
  layer.trainable = False

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 2, 2, 64)          1179712   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 1, 64)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                      

In [13]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=25,
                    verbose=1,
                    )

ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [15]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

NameError: name 'history' is not defined

In [16]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

NameError: name 'history' is not defined